In [1]:
from itertools import product
import os
import pandas as pd
import shutil

In [2]:
# already reduced to presolve < 5000 x 5000
instance_set = "miplib_2017_5000_v2"
max_runtime = 3600
expected_instances = 4
degrees = [-1, 1]
perturbations = ["matrix", "objective", "rhs"]

# Get breakdown of successes and failures by mode

In [3]:
# what I want to know is which error modes account for which incomplete test sets

# running list of strings contained by different error codes
# last two are catchalls
err = {
    "skipping": [],
    f"walltime": [],
    "failed to perturb": [],
    "gurobipy.gurobierror: out of memory": [],
}

warn = {
    "samples of" : [],
    "invalid value encountered in double_scalars": [],
    "could not be solved for": [],
    "presolve failed to reduce": [],
}

# runs that weren't run
no_go = []

# runs that errored out with new error code
other = []

# runs that had no errors
empty = []

# count of instances made
count = {}

for i, file_name in enumerate(os.listdir(os.path.join("instances", instance_set))):
    
    # skip anything not a mip
    if not file_name.endswith(".mps"):
        continue
        
    # get the model name
    stem = file_name[:-4]
    
    # instantiate the count
    count[stem] = {}
    for p, d in product(perturbations, degrees):
        if os.path.isdir(os.path.join("test_sets", instance_set, stem, f"{p}_{d}")):
            count[stem][(p, d)] = len([f for f in os.listdir(os.path.join("test_sets", instance_set, stem, f"{p}_{d}")) if f.endswith(".mps")])
        else:
            count[stem][(p, d)] = 0
    
    # get the error file path
    err_pth = os.path.join("outfiles_generation", stem + ".err")
    
    # check if the series wasn't run
    if not os.path.exists(err_pth):
        no_go.append(stem)
    
    # check if the series ran with no errors or warnings
    elif os.path.getsize(err_pth) == 0:
        empty.append(stem)
        
    else:
        # read in file
        with open(err_pth, "r") as f:
            text = f.read().lower()
            
        # check for error codes
        found_code = False
        for code in err:
            if code in text:
                err[code].append(file_name)
                found_code = True
                break
        
        if not found_code:
            for code in warn:
                if code in text:
                    warn[code].append(file_name)
                    found_code = True
                    break
                    
        if not found_code:
            other.append(file_name)

In [4]:
no_go

[]

In [5]:
# skipping - could not solve base instance in < 1 hour
len(err[f"skipping"]) / (i + 1)

0.22950819672131148

In [6]:
err[f"skipping"]

['seymour.mps',
 'hgms-det.mps',
 'xmas10.mps',
 'stein9inf.mps',
 'dell.mps',
 'misc05inf.mps',
 'fhnw-binpack4-4.mps',
 'milo-v13-4-3d-4-0.mps',
 'cvs08r139-94.mps',
 'fhnw-binpack4-18.mps',
 'control30-5-10-4.mps',
 'neos859080.mps',
 'graph20-20-1rand.mps',
 'neos-1420790.mps',
 'assign1-10-4.mps',
 'neos-5261882-treska.mps',
 'no-ip-65059.mps',
 'lotsize.mps',
 'cvs16r128-89.mps',
 'neos-3530905-gaula.mps',
 'rococoB10-011000.mps',
 'no-ip-64999.mps',
 'neos-3682128-sandon.mps',
 'stein45inf.mps',
 'pb-market-split8-70-4.mps',
 'set3-09.mps',
 'nag.mps',
 'ponderthis0517-inf.mps',
 'neos-3024952-loue.mps',
 'set3-20.mps',
 'markshare2.mps',
 'b2c1s1.mps',
 'gsvm2rl5.mps',
 'neos-2656603-coxs.mps',
 'bppc6-06.mps',
 'neos-3009394-lami.mps',
 'cvrpp-n16k8vrpi.mps',
 'neos-5045105-creuse.mps',
 'queens-30.mps',
 'cvs16r106-72.mps',
 'p2m2p1m1p0n100.mps',
 'sorrell8.mps',
 'milo-v13-4-3d-3-0.mps',
 'bppc6-02.mps',
 'icir97_potential.mps',
 'xmas10-2.mps',
 'stoch-vrpvrp-s5v2c8vrp-v2c8

In [7]:
# perturbation algorithm failed to complete due to not making any perturbations
len(err["failed to perturb"]) / (i + 1)

0.0

In [8]:
# ran out of memory
bumpable = []
print(err["gurobipy.gurobierror: out of memory"])
print(len(err["gurobipy.gurobierror: out of memory"]) / (i + 1))
df = pd.read_csv("more_memory.csv", index_col=0)
for file_name in err["gurobipy.gurobierror: out of memory"]:
    current_memory = df["memory"].get(file_name, 4)  # df.loc[file_name, "memory"]
    df.loc[file_name, "memory"] = min(15, current_memory * 2)
    if current_memory < 15:
        bumpable.append(file_name)
print(len(bumpable) / (i + 1))
# df["memory"].astype(int)
# df.to_csv("more_memory.csv")
# for file_name in bumpable:
#     if os.path.isdir(os.path.join("test_sets", instance_set, file_name[:-4])):
# #        shutil.rmtree(os.path.join("test_sets", instance_set, file_name[:-4]))
#         print("removed: ", file_name[:-4])

['ej.mps', 'neos-3661949-lesse.mps', 'neos-3592146-hawea.mps', 'neos-3627168-kasai.mps', 'gen-ip016.mps']
0.01366120218579235
0.00273224043715847


In [9]:
# less of an issue - perturbation algorithm failed to complete due to time
len(err[f"walltime"]) / (i + 1)

0.00819672131147541

In [10]:
# less of an issue - perturbation algorithm failed to complete due to exhaustion of perturbation attempts
len(warn["samples of"]) / (i + 1)

0.4180327868852459

In [11]:
# nonissue
len(warn["invalid value encountered in double_scalars"]) / (i + 1)

0.0

In [12]:
# nonissue
len(warn["could not be solved for"]) / (i + 1)

0.0

In [13]:
# nonissue
len(warn["presolve failed to reduce"]) / (i + 1)

0.0

In [14]:
len(empty) / (i + 1)

0.33060109289617484

In [15]:
print(other)
len(other) /( i + 1)

[]


0.0

In [17]:
complete = []
removals = []
for stem in count:
    if len([count for (p, d), count in count[stem].items() if count < expected_instances]) == 0:
        complete.append(stem)
    else:
        # remove the directory
        if os.path.isdir(os.path.join("test_sets", instance_set, stem)):
            # shutil.rmtree(os.path.join("test_sets", instance_set, stem))
            removals.append(stem)
len(complete) / (i + 1)

0.3224043715846995

In [18]:
len(removals) / (i + 1)

0.6420765027322405

In [19]:
for stem in removals:
    for (p, d), amt in count[stem].items():
        if amt < expected_instances:
            print(stem, p, d, amt)

seymour matrix -1 0
seymour matrix 1 0
seymour objective -1 0
seymour objective 1 0
seymour rhs -1 0
seymour rhs 1 0
b1c1s1 rhs 1 0
neos-4650160-yukon matrix 1 0
neos-4650160-yukon rhs -1 3
neos-4650160-yukon rhs 1 0
neos-5075914-elvire matrix -1 0
neos-5075914-elvire matrix 1 0
neos-5075914-elvire objective -1 0
neos-5075914-elvire objective 1 0
neos-5075914-elvire rhs -1 0
neos-5075914-elvire rhs 1 0
neos-1396125 matrix 1 2
neos-1396125 rhs -1 3
neos-1396125 rhs 1 0
rocI-4-11 matrix -1 0
rocI-4-11 matrix 1 0
rocI-4-11 rhs 1 0
hgms-det matrix -1 0
hgms-det matrix 1 0
hgms-det objective -1 0
hgms-det objective 1 0
hgms-det rhs -1 0
hgms-det rhs 1 0
pigeon-16 objective 1 0
pigeon-16 rhs 1 0
xmas10 matrix -1 0
xmas10 matrix 1 0
xmas10 objective -1 0
xmas10 objective 1 0
xmas10 rhs -1 0
xmas10 rhs 1 0
neos-4393408-tinui rhs 1 0
piperout-d27 matrix -1 0
piperout-d27 matrix 1 0
piperout-d27 rhs -1 0
piperout-d27 rhs 1 0
stein9inf matrix -1 0
stein9inf matrix 1 0
stein9inf objective -1 0
ste

In [23]:
os.listdir(os.path.join('test_sets', instance_set, "seymour"))

[]